In [1]:
from pulp import LpMinimize, LpProblem, LpStatus, LpVariable, SCIP_PY, SCIP
import pandas as pd

# Parameters used for optimization 
number_of_bronze_dps = 20
max_bronze_dp_size = 5
max_optimization_execution_time = 60 #12*3600

# Read and setup initial values
dependencies = pd.read_csv('data/gold_dp_table_dependencies.csv')
table_names = list(dependencies['table_name'])
gold_dp_names = dependencies.columns[2:].to_list()
gold_dp_indices = range(len(gold_dp_names))
bronze_dp_indices = range(number_of_bronze_dps)
num_rows_per_table = dict(zip(dependencies['table_name'], dependencies['num_rows']))
required_tables_per_gold_dp = {g : set(dependencies[dependencies[g]]['table_name']) for g in gold_dp_names}

# Create the model
model = LpProblem(name="optimise-deps", sense=LpMinimize)

# Variables expressing whether gold_dp i requires bronze_dp j
gb_tuples = [(g,b) for g in gold_dp_indices for b in bronze_dp_indices]
gb_vars = LpVariable.dicts("gold-bronze-deps", gb_tuples, cat="Binary")

# Variables expressing whether gold_dp i requires bronze_dp j
tb_tuples = [(t,b) for t in table_names for b in bronze_dp_indices]
tb_vars = LpVariable.dicts("table-in-bronze_rels", tb_tuples, cat="Binary")

# Variables expressing whether gold_dp i requires table j
gt_tuples = [(g,t) for g in gold_dp_indices for t in table_names]
gt_vars = LpVariable.dicts("gold-table-deps", gt_tuples, cat="Binary")

# Helper variables expressing whether gold_dp i obtains table j through bronze_dp k
h_gbt_tuples = [(g,b,t) for g in gold_dp_indices for b in bronze_dp_indices for t in table_names]
h_gbt_vars = LpVariable.dicts("helper-vars-table-via-bronze-in-gold", h_gbt_tuples, cat="Binary")

# Helper variables expressing whether gold_dp i obtains table j
h_gt_tuples = [(g,t) for g in gold_dp_indices for t in table_names]
h_gt_vars = LpVariable.dicts("helper-vars-gold-contains-table", h_gt_tuples, cat="Binary")

# Setup helper variables
for gold_dp in gold_dp_indices:
        for table in table_names:
            for bronze_dp in bronze_dp_indices:
                model += h_gbt_vars[gold_dp, bronze_dp, table] <= gb_vars[gold_dp, bronze_dp]
                model += h_gbt_vars[gold_dp, bronze_dp, table] <= tb_vars[table, bronze_dp]
                model += h_gbt_vars[gold_dp, bronze_dp, table] >= gb_vars[gold_dp, bronze_dp] + tb_vars[table, bronze_dp] - 1
                model += h_gt_vars[gold_dp,table] >= h_gbt_vars[gold_dp, bronze_dp, table]

# Setup gt_vars by according to dependencies in input file
for gold_dp in gold_dp_indices:
    req_tables = list(dependencies[dependencies[gold_dp_names[gold_dp]]]['table_name'])
    for table in table_names:
        if table in req_tables:
            model += gt_vars[gold_dp, table] == 1
        else:
            model += gt_vars[gold_dp, table] == 0

# Setup requirement for maximum size of bronze dps
for bronze_dp in bronze_dp_indices:
    expr = 0
    for table in table_names:
        expr += tb_vars[table, bronze_dp]
    model += expr <= max_bronze_dp_size

# Setup requirement for gold dps to obtain all their dependencies
for gold_dp in gold_dp_indices:
        for table in table_names:
            expr = 0
            for bronze_dp in bronze_dp_indices:
                expr += h_gbt_vars[gold_dp, bronze_dp, table]
            if table in required_tables_per_gold_dp[gold_dp_names[gold_dp]]:
                model += expr >= 1

# Setup requirement to have each table in a single bronze dp only
for table in table_names:
    expr = 0
    for bronze_dp in bronze_dp_indices:
        expr += tb_vars[table, bronze_dp]
    model += expr <= 1

# Setup goal to minimize unnecessarily loaded table rows
optimization_expr_per_gold_dp = {}
optimization_expr = 0
for gold_dp in gold_dp_indices:
        optimization_expr_per_gold_dp[gold_dp] = 0
        for table in table_names:
            optimization_expr_per_gold_dp[gold_dp] += -gt_vars[gold_dp, table]*num_rows_per_table[table]
            optimization_expr_per_gold_dp[gold_dp] += h_gt_vars[gold_dp, table]*num_rows_per_table[table]
        optimization_expr += optimization_expr_per_gold_dp[gold_dp]

model += optimization_expr

# Solve the problem
solver = SCIP(timeLimit=max_optimization_execution_time)
status = model.solve(solver)

# Print result
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

SCIP version 10.0.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: SoPlex 8.0.0] [GitHash: 0c80fdd8e9]
Copyright (c) 2002-2025 Zuse Institute Berlin (ZIB)

External libraries: 
  SoPlex 8.0.0         Linear programming solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 2207cfb2]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  ZLIB 1.3.1           General purpose compression library by J. Gailly and M. Adler (zlib.net)
  MPFR 4.2.2           GNU Multiple Precision Floating-Point Reliable Library (mpfr.org)
  Boost 1.83.0         Boost C++ Libraries (boost.org)
  TinyCThread 1.2      small portable implementation of the C11 threads API (tinycthread.github.io)
  GMP 6.3.0            GNU Multiple Precision Arithmetic Library developed by T. Granlund (gmplib.org)
  ZIMPL 3.7.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 4.0.3 

In [2]:
# Create dataframes to visualize optimal relations

gold_bronze_dependencies = pd.DataFrame(index=bronze_dp_indices, columns=gold_dp_indices)
for gold_dp in gold_dp_indices:
    for bronze_dp in bronze_dp_indices:
        if gb_vars[gold_dp, bronze_dp].varValue >= 1:
            gold_bronze_dependencies.at[bronze_dp, gold_dp] = True
        else:
            gold_bronze_dependencies.at[bronze_dp, gold_dp] = False

bronze_table_relations = pd.DataFrame(index=table_names, columns=bronze_dp_indices)
for bronze_dp in bronze_dp_indices:
    for table in table_names:
        if tb_vars[table, bronze_dp].varValue >= 1:
            bronze_table_relations.at[table, bronze_dp] = True
        else:
            bronze_table_relations.at[table, bronze_dp] = False

In [3]:
# Evaluate current dependencies of gold products

data_rows = []

for gold_dp in gold_dp_indices:
    bronze_dps_per_gold_dp = list(gold_bronze_dependencies[gold_bronze_dependencies[gold_dp]].index)
    available_tables_per_gold_dp = list(bronze_table_relations[bronze_table_relations[bronze_dps_per_gold_dp].any(axis=1)].index)
    
    req_tables = dependencies[dependencies[gold_dp_names[gold_dp]]]
    available_tables = dependencies[dependencies['table_name'].isin(available_tables_per_gold_dp)]
    req_rows = req_tables['num_rows'].sum()
    available_rows = available_tables['num_rows'].sum()
    req_table_count = len(req_tables)
    available_table_count = len(available_tables)
    

    data_rows.append({
        'gold_product': gold_dp_names[gold_dp], 
        'Number of required tables': req_table_count, 
        'Number of available tables': available_table_count,
        'Number of required data rows': req_rows, 
        'Number of available data rows': available_rows,
        'Ratio of required data over available data': 0 if available_rows == 0 else req_rows/available_rows,
    })

gold_dependencies_analysis = pd.DataFrame(data_rows)

In [ ]:
# Export Summary to Excel

with pd.ExcelWriter('data/optimization_summary.xlsx', engine='openpyxl') as writer:
    dependencies.to_excel(writer, sheet_name='Dependencies Gold DP to Tables', index=False)
    gold_bronze_dependencies.to_excel(writer, sheet_name='Mapping Bronze DP to Gold DP')
    bronze_table_relations.to_excel(writer, sheet_name='Mapping Table to Bronze DP')
    gold_dependencies_analysis.to_excel(writer, sheet_name='Validation')